# 作成した各種関数などをまとめるノート

In [1]:
import glob
import japanize_matplotlib
import os
import pandas as pd
import random
import sys
import math
import matplotlib as mpl
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import sklearn.preprocessing as sp
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [2]:
# 各種値を宣言
benchmarks = ['bt', 'cg', 'ep', 'ft', 'is', 'lu', 'mg', 'sp']
classes = ["S", "W", "A", "B", "C", "D"]
processes = [1, 2, 4, 8, 16, 32, 64, 128, 256]
fix_process = 4
fix_benchmark_class = "C"

In [3]:
def return_fixed_class(BenchMark="bt", Processes=[1, 2, 4, 8, 16, 32, 64, 128, 256], FixedBenchMarkClass="C"):
        path = './csv_files/'
        
        # fixed_Class にはベンチマーククラスFixedBenchMarkClassで実行プロセス数がProcessesに該当するものの結果が入る
        fixed_Class = list()

        for process in Processes:
            file_name = ("pprof_"+BenchMark+FixedBenchMarkClass+str(process)+".csv")
            file_path = path+file_name
            if (os.path.exists(file_path) and os.stat(file_path).st_size != 0):
                data_frame = pd.read_csv(path+file_name)
                data_frame = data_frame.set_index(['Name'])
                fixed_Class.append(data_frame.rename(columns = {'#Call': process}).sort_index())
        return(fixed_Class)
    

In [4]:
def show_fixed_class_graph(BenchMark="bt", Processes=[1, 2, 4, 8, 16, 32, 64, 128, 256], FixedBenchMarkClass="C"):

        markers = [".", ",", "o", "v", "^", "<", ">", "1", "2", "3", "4", "8", "s", "p", "*", "h", "H", "+", "x", "D", "d", "|", "_", "None", None, "", "$x$",
            "$\\alpha$", "$\\beta$", "$\\gamma$"]
        colors = ['#e41a1c', '#377eb8', '#4daf4a', '#984ea3', '#ff7f00', '#ffff33', '#a65628', '#f781bf'] 
        fixed_Class = return_fixed_class(BenchMark=BenchMark, Processes=Processes, FixedBenchMarkClass=FixedBenchMarkClass)
        if(len(fixed_Class) != 0):
            summary_fixed_Class = pd.concat(fixed_Class, axis=1)
            summary_fixed_Class.sort_index(axis=1, inplace=True)
            summary_fixed_Class_title = BenchMark+"においてベンチマーククラスをCに固定し実行プロセス数を変化させたときの実行された全ての関数のコール回数"

            x_axes = summary_fixed_Class.columns.tolist()

            y_axes = summary_fixed_Class.index.tolist()

            plt.figure()
            for y_axis in y_axes:
                color = random.choice(colors)
                label = y_axis
                marker = random.choice(markers)
                plt.plot(x_axes, summary_fixed_Class.T[y_axis], marker=marker, label=y_axis)
            plt.legend()
            plt.title(BenchMark+"_FixedBenchMarkClass="+FixedBenchMarkClass)
            plt.show()
            
# 使用例
# show_fixed_class_graph(BenchMark="cg")

In [5]:
def return_fixed_process(BenchMark="bt", BenchMarkClasses=["S", "W", "A", "B", "C", "D"], FixedProcess=32):
    path = './csv_files/'

    # fixed_process には実行プロセス数が64でベンチマーククラスがA ~ Dまでの結果が入る
    fixed_process = list()

    for bench_mark_class in BenchMarkClasses:
        file_name = ("pprof_"+BenchMark+bench_mark_class+str(FixedProcess)+".csv")
        file_path = path+file_name
        if (os.path.exists(file_path) and os.stat(file_path).st_size != 0):
            data_frame = pd.read_csv(path+file_name)
            data_frame = data_frame.set_index(['Name'])
            fixed_process.append(data_frame.rename(columns = {'#Call': bench_mark_class}).sort_index())
    return(fixed_process)
       

In [6]:
     
def show_fixed_process_graph(BenchMark="bt", BenchMarkClasses=["S", "W", "A", "B", "C", "D"], FixedProcess=32):
    fixed_process = return_fixed_process(BenchMark=BenchMark, BenchMarkClasses=BenchMarkClasses, FixedProcess=FixedProcess)
    markers = [".", ",", "o", "v", "^", "<", ">", "1", "2", "3", "4", "8", "s", "p", "*", "h", "H", "+", "x", "D", "d", "|", "_", "None", None, "", "$x$",
 "$\\alpha$", "$\\beta$", "$\\gamma$"]
    colors = ['#e41a1c', '#377eb8', '#4daf4a', '#984ea3', '#ff7f00', '#ffff33', '#a65628', '#f781bf']
    if (len(fixed_process) != 0):
            summary_fixed_process = pd.concat(fixed_process, axis=1)

            x_axes = summary_fixed_process.columns.tolist()

            y_axes = summary_fixed_process.index.tolist()

            plt.figure()
            for y_axis in y_axes:
                color = random.choice(colors)
                label = y_axis
                marker = random.choice(markers)
                plt.plot(x_axes, summary_fixed_process.T[y_axis], marker=marker, label=y_axis)
            plt.legend()
            plt.title(BenchMark+"_FixedProcess="+str(FixedProcess))
            plt.show()
            
# 使用例            
# show_fixed_process_graph(BenchMark="cg")

In [7]:
def show_graph(BenchMarks=[], Processes=[], BenchMarkClasses=[], fix_process=4, fix_benchmark_class="C"):

    if (BenchMarks==[] or Processes==[] or BenchMarkClasses==[]):
        print("関数の引数となっている配列が空です。")
    else:
        for bench_mark in BenchMarks:
            show_fixed_class_graph(BenchMark=bench_mark, Processes=Processes, FixedBenchMarkClass=fix_benchmark_class)
            show_fixed_process_graph(BenchMark=bench_mark, BenchMarkClasses=BenchMarkClasses, FixedProcess=fix_process)            

bench_marks = ['bt', 'cg', 'ep', 'ft', 'is', 'lu', 'mg', 'sp']
processes = [1, 2, 4, 8, 16, 32, 64, 128, 256]
classes = ["S", "W", "A", "B", "C", "D"]

# 使用例
# show_graph(bench_marks, processes, classes)

In [8]:
def return_summarized_Fixed_dataframe(BenchMark_name = "cg", fixed="class"):

    def evaluate_dataframes(df1, df2):
        for i in range(len(df1.values.tolist()[0])):
            if(df1.values.tolist()[0][i] != df2.values.tolist()[0][i]):
                return False
        return True
    fixed_df = 0
    if (fixed == "class"):
        fixed_df = return_fixed_class(BenchMark=BenchMark_name)
    elif (fixed == "process"):
        fixed_df = return_fixed_process(BenchMark=BenchMark_name)
    summary_fixed_df = pd.concat(fixed_df, axis=1)
    dropped_summary_fixed_df = summary_fixed_df.drop_duplicates()
    dropped_summary_fixed_df_renamed = dropped_summary_fixed_df

    for dropped_index_name in dropped_summary_fixed_df.index.values:
        dropped_index_name_data = summary_fixed_df.loc[[dropped_index_name]]
        replace_name = dropped_index_name
        for all_index_name in summary_fixed_df.index.values:
            all_index_name_data = summary_fixed_df.loc[[all_index_name]]
            if(dropped_index_name == all_index_name):
                pass
            elif(evaluate_dataframes(dropped_index_name_data, all_index_name_data)):
                replace_name += f", {all_index_name}"
        dropped_summary_fixed_df_renamed = dropped_summary_fixed_df_renamed.rename(index={dropped_index_name: replace_name})
    
    return dropped_summary_fixed_df_renamed
    

In [9]:
class ModelBase:
    def __init__(self, raw_x, raw_y, benchmark_name="benchmark_name", function_name="function_name"):
        self.benchmark_name = benchmark_name
        self.function_name = function_name
        self.xlabel = "実行時のプロセス数"
        self.ylabel = "関数のコール回数"

        test_rate = math.floor(len(raw_x) * 0.3)
        self.raw_x = np.reshape(raw_x, (-1, 1))
        self.raw_y = np.reshape(raw_y, (-1, 1))
        self.train_x = self.raw_x[:-test_rate]
        self.train_y = self.raw_y[:-test_rate]
        self.test_x = self.raw_x[-test_rate:]
        self.test_y = self.raw_y[-test_rate:]
        
        self.x_model_line = np.random.rand(self.raw_x.max(), 1) * self.raw_x.max()
        self.x_model_line.sort(axis=0)
        self.y_model_line = 0
        
        self.lr = 0
        self.r2_score = 0
        
    def calc_lr(self):
        self.lr = 0
     
    def calc_r2_score(self):
        self.r2_score = 0
        
    def calc_mae_score(self):
        self.mae_score = 0
        
    def calc_mse_score(self):
        self.mse_score = 0
        
    def plot_graph(self):
        plt.figure()
        plt.plot(self.raw_x, self.raw_y)
        plt.show()
        

In [10]:
class ModelLin(ModelBase):
    def calc_lr(self):
        self.lr = LinearRegression()
        self.lr.fit(self.train_x, self.train_y)
        
    def calc_r2_score(self):
        raw_y_predicted = self.lr.predict(self.raw_x)
        self.r2_score = r2_score(self.raw_y, raw_y_predicted)
        
    def calc_mae_score(self):
        raw_y_predicted = self.lr.predict(self.raw_x)
        self.mae_score = mean_absolute_error(self.raw_y, raw_y_predicted)
        
    def calc_mse_score(self):
        raw_y_predicted = self.lr.predict(self.raw_x)
        self.mse_score = mean_squared_error(self.raw_y, raw_y_predicted)
        
    def plot_graph(self):
        plt.figure()
        plt.scatter(self.raw_x, self.raw_y)
        self.y_model_line = self.lr.predict(self.x_model_line)
        plt.plot(self.x_model_line, self.y_model_line)
        plt.title(f"ベンチマーク名：{self.benchmark_name}, 関数名：{self.function_name}", y=-0.2)
        plt.xlabel = self.xlabel
        plt.ylabel = self.ylabel
        plt.show()
        

In [11]:
def inverter_log10_func(x):
    return 10**x

class ModelLog10(ModelBase):
    
    def calc_lr(self):
        self.transformer_log10 = sp.FunctionTransformer(func=np.log10, inverse_func=inverter_log10_func)
        x_train_log10 = self.transformer_log10.transform(self.train_x)
        y_train_log10 = self.transformer_log10.transform(self.train_y)
        
        self.lr = LinearRegression()
        self.lr.fit(x_train_log10, y_train_log10)
        
    def calc_r2_score(self):
        raw_x_log10 = self.transformer_log10.transform(self.raw_x)
        raw_y_predicted_log10 = self.lr.predict(raw_x_log10)
        raw_y_predicted = self.transformer_log10.inverse_transform(raw_y_predicted_log10)
        self.r2_score = r2_score(self.raw_y, raw_y_predicted)
        
    def calc_mae_score(self):
        raw_x_log10 = self.transformer_log10.transform(self.raw_x)
        raw_y_predicted_log10 = self.lr.predict(raw_x_log10)
        raw_y_predicted = self.transformer_log10.inverse_transform(raw_y_predicted_log10)
        self.mae_score = mean_absolute_error(self.raw_y, raw_y_predicted)
        
    def calc_mse_score(self):
        raw_x_log10 = self.transformer_log10.transform(self.raw_x)
        raw_y_predicted_log10 = self.lr.predict(raw_x_log10)
        raw_y_predicted = self.transformer_log10.inverse_transform(raw_y_predicted_log10)
        self.mse_score = mean_squared_error(self.raw_y, raw_y_predicted)
        
    def plot_graph(self):
        plt.figure()
        plt.scatter(self.raw_x, self.raw_y)
        x_model_line_log10 = self.transformer_log10.transform(self.x_model_line)
        y_model_line_log10 = self.lr.predict(x_model_line_log10)
        self.y_model_line = self.transformer_log10.inverse_transform(y_model_line_log10)
        plt.plot(self.x_model_line, self.y_model_line)
        plt.title(f"ベンチマーク名：{self.benchmark_name}, 関数名：{self.function_name}", y=-0.2)
        plt.xlabel = self.xlabel
        plt.ylabel = self.ylabel
        plt.show()
        

In [12]:
# Inverse proportion
def ip_func(x):
    return 1/x

class ModelIP(ModelBase):
    
    def calc_lr(self):
        self.transformer_ip = sp.FunctionTransformer(func=ip_func, inverse_func=ip_func)
        y_train_ip = self.transformer_ip.transform(self.train_y)        
        self.lr = LinearRegression()
        self.lr.fit(self.train_x, y_train_ip)
        
    def calc_r2_score(self):
        raw_y_predicted_ip = self.lr.predict(self.raw_x)
        raw_y_predicted = self.transformer_ip.inverse_transform(raw_y_predicted_ip)
        self.r2_score = r2_score(self.raw_y, raw_y_predicted)
        
    def calc_mae_score(self):
        raw_y_predicted_ip = self.lr.predict(self.raw_x)
        raw_y_predicted = self.transformer_ip.inverse_transform(raw_y_predicted_ip)
        self.mae_score = mean_absolute_error(self.raw_y, raw_y_predicted)
        
    def calc_mse_score(self):
        raw_y_predicted_ip = self.lr.predict(self.raw_x)
        raw_y_predicted = self.transformer_ip.inverse_transform(raw_y_predicted_ip)
        self.mse_score = mean_squared_error(self.raw_y, raw_y_predicted)
        
    def plot_graph(self):
        plt.figure()
        plt.scatter(self.raw_x, self.raw_y)
        y_model_line_ip = self.lr.predict(self.x_model_line)
        self.y_model_line = self.transformer_ip.inverse_transform(y_model_line_ip)
        plt.plot(self.x_model_line, self.y_model_line)
        plt.title(f"ベンチマーク名：{self.benchmark_name}, 関数名：{self.function_name}", y=-0.2)
        plt.xlabel = self.xlabel
        plt.ylabel = self.ylabel
        plt.show()
        